In [20]:
import pandas as pd
import numpy as np
import re
import string
import os
import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
init_notebook_mode(connected=True)


In [21]:
df = pd.read_csv('zomato.csv', index_col=0)

In [22]:
df.head()

,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
Restaurant ID,,,,,,,,,,,,,,,,,,,,
6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [23]:
# avg cost for two and restaurant rating

    Attach country to country code

In [24]:
country_map = {'1':'India',
'14':'Australia',
'30':'Brazil',
'37':'Canada',
'94':'Indonesia',
'148':'New Zealand',
'162':'Phillipines',
'166':'Qatar',
'184':'Singapore',
'189':'South Africa',
'191':'Sri Lanka',
'208':'Turkey',
'214':'UAE',
'215':'United Kingdom',
'216':'United States'}

df['country'] = df['Country Code'].apply(lambda x: country_map[str(x)])

    Get cost in usd

In [62]:
def convert_to_usd(df, col_name):
    if df['country'] == 'Australia' : return df[col_name]*0.019
    if df['country'] == 'Brazil' : return df[col_name]*0.29
    if df['country'] == 'Canada' : return df[col_name]*0.78
    if df['country'] == 'Indonesia' : return df[col_name]*0.000072
    if df['country'] == 'New Zealand' : return df[col_name]*0.72
    if df['country'] == 'Phillipines' : return df[col_name]*0.019
    if df['country'] == 'Qatar' : return df[col_name]*0.27
    if df['country'] == 'Singapore' : return df[col_name]*0.76
    if df['country'] == 'South Africa' : return df[col_name]*0.083
    if df['country'] == 'Sri Lanka' : return df[col_name]*0.0064
    if df['country'] == 'Turkey' : return df[col_name]*0.25
    if df['country'] == 'UAE' : return df[col_name]*0.27 
    if df['country'] == 'United Kingdom' : return df[col_name]*1.40
    if df['country'] == 'United States' : return df[col_name]*1.0
    if df['country'] == 'India' : return df[col_name]*0.015 

In [63]:
df['cost_for_two_usd'] = df.apply(lambda x: convert_to_usd(x, 'Average Cost for two'), axis=1)

    Get primary cuisine for each restaurant

In [27]:
df['primary_cuisine'] = df['Cuisines'].apply(lambda x: str(x).split(',')[0])

    Top 7 cuisine

In [28]:
# popular 7 cuisines
list(pd.DataFrame(df.primary_cuisine.value_counts()[:7]).index)

['North Indian',
 'Chinese',
 'Fast Food',
 'Bakery',
 'Cafe',
 'American',
 'South Indian']

    Draw for only these top 7 cuisines

In [180]:
df_new = df[df.primary_cuisine.isin(list(pd.DataFrame(df.primary_cuisine.value_counts()[:7]).index))]

Get only the required columns

In [181]:
df_new = df_new[['cost_for_two_usd', 'primary_cuisine', 'Price range', 'Aggregate rating', 'Votes']]

In [182]:
df_new['Votes'] = (df_new['Votes']/1000).astype('int')*1000

In [183]:
df_new['Votes'].unique()

array([   0, 1000, 3000, 2000, 9000, 5000, 4000])

In [152]:
# df_new['cost_for_two_usd'] = df_new['cost_for_two_usd'].astype('int')
# df_new['Aggregate rating'] = df_new['Aggregate rating'].astype('int')

In [153]:
# sizeref = 2. * max(df_new['Votes']) / (100 ** 2)

In [154]:
# trace0 = go.Scatter(
#     x=list(df_new.cost_for_two_usd.values),
#     y=list(df_new['Aggregate rating'].values),
#     mode='markers',
#    marker=dict(
#         sizemode='area',
#         size=df_new['Votes'],
#         sizeref=sizeref,
#         line=dict(
#             width=2
#         ),
#    )
# )

In [155]:
# data = [trace0]

In [156]:
# iplot(data)

For each cuisine

In [270]:
df_new.columns

Index([u'cost_for_two_usd', u'primary_cuisine', u'Price range',
       u'Aggregate rating', u'Votes'],
      dtype='object')

In [280]:
sizeref = 2. * max(df_new['Votes']) / (100 ** 2)

In [281]:
hover_text = []
for index, row in df_new.iterrows():
    hover_text.append(('Cuisine: {Cuisine}<br>'+
                      'Rating: {Rating}<br>'+
                      'Votes: {Votes}<br>'+
                      'Price: {Price}<br>'+
                      'Cost_Two: {Cost_Two}').format(Cuisine=row['primary_cuisine'],
                                            Rating=row['Price range'],
                                            Price=row['Aggregate rating'],
                                            Votes=row['Votes'],
                                            Cost_Two=row['cost_for_two_usd']))
    
df_new['text'] = hover_text

In [285]:
trace0 = go.Scatter(
    x=df_new['cost_for_two_usd'][df_new['primary_cuisine'] == 'Chinese'][:50],
    y=df_new['Aggregate rating'][df_new['primary_cuisine'] == 'Chinese'][:50],
    mode='markers',
    name='Chinese',
    text=df_new['text'][df_new['primary_cuisine'] == 'Chinese'],
    marker=dict(
        symbol='circle',
        sizemode='area',
        sizeref=sizeref,
        size=df_new['Votes'][df_new['primary_cuisine'] == 'Chinese'][:50],
        line=dict(
            width=2
        ),
    )
)

trace1 = go.Scatter(
    x=df_new['cost_for_two_usd'][df_new['primary_cuisine'] == 'American'][:50],
    y=df_new['Aggregate rating'][df_new['primary_cuisine'] == 'American'][:50],
    mode='markers',
    name='American',
    text=df_new['text'][df_new['primary_cuisine'] == 'American'],
    marker=dict(
        symbol='circle',
        sizemode='area',
        sizeref=sizeref,
        size=df_new['Votes'][df_new['primary_cuisine'] == 'American'][:50],
        line=dict(
            width=2
        ),
    )
)


trace2 = go.Scatter(
    x=df_new['cost_for_two_usd'][df_new['primary_cuisine'] == 'Cafe'][:50],
    y=df_new['Aggregate rating'][df_new['primary_cuisine'] == 'Cafe'][:50],
    mode='markers',
    name='Cafe',
    text=df_new['text'][df_new['primary_cuisine'] == 'Cafe'],
    marker=dict(
        symbol='circle',
        sizemode='area',
        sizeref=sizeref,
        size=df_new['Votes'][df_new['primary_cuisine'] == 'Cafe'][:50],
        line=dict(
            width=2
        ),
    )
)

trace3 = go.Scatter(
    x=df_new['cost_for_two_usd'][df_new['primary_cuisine'] == 'Fast Food'][:50],
    y=df_new['Aggregate rating'][df_new['primary_cuisine'] == 'Fast Food'][:50],
    mode='markers',
    name='Fast Food',
    text=df_new['text'][df_new['primary_cuisine'] == 'Fast Food'],
    marker=dict(
        symbol='circle',
        sizemode='area',
        sizeref=sizeref,
        size=df_new['Votes'][df_new['primary_cuisine'] == 'Fast Food'][:50],
        line=dict(
            width=2
        ),
    )
)

trace4 = go.Scatter(
    x=df_new['cost_for_two_usd'][df_new['primary_cuisine'] == 'Bakery'][:50],
    y=df_new['Aggregate rating'][df_new['primary_cuisine'] == 'Bakery'][:50],
    mode='markers',
    name='Bakery',
    text=df_new['text'][df_new['primary_cuisine'] == 'Bakery'],
    marker=dict(
        symbol='circle',
        sizemode='area',
        sizeref=sizeref,
        size=df_new['Votes'][df_new['primary_cuisine'] == 'Bakery'][:50],
        line=dict(
            width=2
        ),
    )
)

trace5 = go.Scatter(
    x=df_new['cost_for_two_usd'][df_new['primary_cuisine'] == 'South Indian'][:50],
    y=df_new['Aggregate rating'][df_new['primary_cuisine'] == 'South Indian'][:50],
    mode='markers',
    name='South Indian',
    text=df_new['text'][df_new['primary_cuisine'] == 'South Indian'],
    marker=dict(
        symbol='circle',
        sizemode='area',
        sizeref=sizeref,
        size=df_new['Votes'][df_new['primary_cuisine'] == 'South Indian'][:50],
        line=dict(
            width=2
        ),
    )
)

trace6 = go.Scatter(
    x=df_new['cost_for_two_usd'][df_new['primary_cuisine'] == 'North Indian'][:50],
    y=df_new['Aggregate rating'][df_new['primary_cuisine'] == 'North Indian'][:50],
    mode='markers',
    name='North Indian',
    text=df_new['text'][df_new['primary_cuisine'] == 'North Indian'],
    marker=dict(
        symbol='circle',
        sizemode='area',
        sizeref=sizeref,
        size=df_new['Votes'][df_new['primary_cuisine'] == 'North Indian'][:50],
        line=dict(
            width=2
        ),
    )
)

In [286]:
data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6]
# data = [trace0, trace1]

In [287]:
iplot(data)

With aggregates

In [199]:
df_new2 = df[df.primary_cuisine.isin(list(pd.DataFrame(df.primary_cuisine.value_counts()[:7]).index))]

In [200]:
df_new2 = df_new2[['cost_for_two_usd', 'primary_cuisine', 'Price range', 'Aggregate rating', 'Votes']]

In [203]:
df_agg = pd.DataFrame(df_new2.groupby('primary_cuisine')[['cost_for_two_usd', 'Price range', 'Aggregate rating', 'Votes']].mean()).reset_index()

In [208]:
df_agg[[u'cost_for_two_usd', u'Price range',
       u'Aggregate rating', u'Votes']] = df_agg[[u'cost_for_two_usd', u'Price range',
       u'Aggregate rating', u'Votes']].astype('int')

In [228]:
df_agg

,primary_cuisine,cost_for_two_usd,Price range,Aggregate rating,Votes
0,American,24,2,3,379
1,Bakery,5,1,2,67
2,Cafe,11,1,3,188
3,Chinese,10,1,2,99
4,Fast Food,5,1,2,63
5,North Indian,9,1,2,119
6,South Indian,6,1,2,131


In [210]:
sizeref = 2. * max(df_agg['Votes']) / (100 ** 2)

In [268]:
trace0 = go.Scatter(
    x=list(df_agg.cost_for_two_usd.values),
    y=list(df_agg['Aggregate rating'].values),
    mode='markers',
    name=['American', 'Bakery', 'Cafe', 'Chinese', 'Fast Food',
       'North Indian', 'South Indian'],
    text=['American', 'Bakery', 'Cafe', 'Chinese', 'Fast Food',
       'North Indian', 'South Indian'],
   marker=dict(
        sizemode='area',
        
        size=df_agg['Votes'],
        color=['rgb(127,201,127)', 'rgb(190,174,212)',  'rgb(253,192,134)', 'rgb(255,255,153)',
              'rgb(56,108,176)', 'rgb(240,2,127)',  'rgb(191,91,23)'],
        sizeref=sizeref,
        
        line=dict(
            width=2
        ),
   )
)

In [269]:
data = [trace0]
iplot(data)